In [1]:
import pandas as pd
import dask.dataframe as dd
import time
import yaml

In [2]:
path = '/Users/home/Medium_Trans.csv'

In [3]:
panda_start = time.time()
panda_data = pd.read_csv(path)
panda_end = time.time()
panda_time = panda_end - panda_start
print(f'Pandas took {panda_time} to read.')

Pandas took 86.02287101745605 to read.


In [4]:
dask_start = time.time()
dask_data = dd.read_csv(path)
dask_end = time.time()
dask_time = dask_end - dask_start
print(f'Pandas took {dask_time} to read.')

Pandas took 0.2665121555328369 to read.


In [5]:
panda_data.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:15,20,800104D70,20,800104D70,8095.07,US Dollar,8095.07,US Dollar,Reinvestment,0
1,2022/09/01 00:18,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:23,1208,80010E430,1208,80010E430,2654.22,US Dollar,2654.22,US Dollar,Reinvestment,0
3,2022/09/01 00:19,3203,80010EA80,3203,80010EA80,13284.41,US Dollar,13284.41,US Dollar,Reinvestment,0
4,2022/09/01 00:27,20,800104D20,20,800104D20,9.72,US Dollar,9.72,US Dollar,Reinvestment,0


In [6]:
dask_data.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:15,20,800104D70,20,800104D70,8095.07,US Dollar,8095.07,US Dollar,Reinvestment,0
1,2022/09/01 00:18,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:23,1208,80010E430,1208,80010E430,2654.22,US Dollar,2654.22,US Dollar,Reinvestment,0
3,2022/09/01 00:19,3203,80010EA80,3203,80010EA80,13284.41,US Dollar,13284.41,US Dollar,Reinvestment,0
4,2022/09/01 00:27,20,800104D20,20,800104D20,9.72,US Dollar,9.72,US Dollar,Reinvestment,0


In [7]:
panda_data.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [8]:
dask_data.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [9]:
%%writefile utility.py

import yaml
import pandas as od
import re
import dask.dataframe as dd
import logging



def read_config_file(a_file):
    with open(a_file, 'r') as stream:
        try:
            return yaml.safe_load(stream) 
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
#     expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
#     df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [10]:
%%writefile file.yaml

file_type: csv
dataset_name: file
file_name: Medium_Trans
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
   - timestamp
   - from_Bank
   - account
   - to_bank
   - account_1
   - amount_received
   - receiving_currency
   - amount_paid
   - payment_currency
   - payment_format
   - is_laundering
     

Overwriting file.yaml


In [11]:
import utility as util

In [12]:
config_data = util.read_config_file('file.yaml')
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Medium_Trans',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['timestamp',
  'from_Bank',
  'account',
  'to_bank',
  'account_1',
  'amount_received',
  'receiving_currency',
  'amount_paid',
  'payment_currency',
  'payment_format',
  'is_laundering']}

In [13]:
f_name = config_data['file_name']
f_name

'Medium_Trans'

In [14]:
f_type = config_data['file_type']
f_type

'csv'

In [15]:
f_delim = config_data['inbound_delimiter']
f_delim

','

In [16]:
data = dd.read_csv(f'{f_name}.{f_type}')
data.tail()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
341574,2022/09/16 23:46,1216195,84FDBD5D1,2213357,850556A01,0.992614,Bitcoin,0.992614,Bitcoin,Bitcoin,0
341575,2022/09/16 23:43,1216195,84FDBD5D1,2213357,850556A01,1.962109,Bitcoin,1.962109,Bitcoin,Bitcoin,0
341576,2022/09/16 23:31,1216195,84FDBD5D1,2213357,850556A01,0.149850,Bitcoin,0.149850,Bitcoin,Bitcoin,0
341577,2022/09/16 23:57,1216195,84FDBD5D1,2213357,850556A01,0.025608,Bitcoin,0.025608,Bitcoin,Bitcoin,0
341578,2022/09/16 23:57,3212631,85057D081,2211696,85057E351,0.019455,Bitcoin,0.019455,Bitcoin,Bitcoin,0


In [18]:
datas = dd.read_csv('covid.csv')
datas.tail()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
270,18/03/2020,18,3,2020,1,0,Kenya,KE,KEN,52573967,Africa,NaN
271,17/03/2020,17,3,2020,0,0,Kenya,KE,KEN,52573967,Africa,NaN
272,16/03/2020,16,3,2020,2,0,Kenya,KE,KEN,52573967,Africa,NaN
273,15/03/2020,15,3,2020,0,0,Kenya,KE,KEN,52573967,Africa,NaN
274,14/03/2020,14,3,2020,1,0,Kenya,KE,KEN,52573967,Africa,NaN


In [19]:
util.col_header_val(datas, config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['month', 'year', 'daterep', 'cases', 'countriesandterritories', 'geoid', 'countryterritorycode', 'popdata2019', 'continentexp', 'day', 'deaths', 'cumulative_number_for_14_days_of_covid_19_cases_per_100000']
Following YAML columns are not in the file uploaded ['to_bank', 'account_1', 'amount_paid', 'payment_format', 'from_bank', 'receiving_currency', 'timestamp', 'payment_currency', 'amount_received', 'is_laundering', 'account']


0

In [20]:
util.col_header_val(data, config_data)

column name and column length validation passed


1

In [21]:
import csv
import gzip
import os

In [22]:
data.columns

Index(['timestamp', 'from_bank', 'account', 'to_bank', 'account_1',
       'amount_received', 'receiving_currency', 'amount_paid',
       'payment_currency', 'payment_format', 'is_laundering'],
      dtype='object')

In [23]:
# Write csv in gz format in pipe separated text file (|)
data.to_csv('Medium_Trans.csv.gz',sep='|',compression='gzip')

['/Users/home/Medium_Trans.csv.gz/00.part',
 '/Users/home/Medium_Trans.csv.gz/01.part',
 '/Users/home/Medium_Trans.csv.gz/02.part',
 '/Users/home/Medium_Trans.csv.gz/03.part',
 '/Users/home/Medium_Trans.csv.gz/04.part',
 '/Users/home/Medium_Trans.csv.gz/05.part',
 '/Users/home/Medium_Trans.csv.gz/06.part',
 '/Users/home/Medium_Trans.csv.gz/07.part',
 '/Users/home/Medium_Trans.csv.gz/08.part',
 '/Users/home/Medium_Trans.csv.gz/09.part',
 '/Users/home/Medium_Trans.csv.gz/10.part',
 '/Users/home/Medium_Trans.csv.gz/11.part',
 '/Users/home/Medium_Trans.csv.gz/12.part',
 '/Users/home/Medium_Trans.csv.gz/13.part',
 '/Users/home/Medium_Trans.csv.gz/14.part',
 '/Users/home/Medium_Trans.csv.gz/15.part',
 '/Users/home/Medium_Trans.csv.gz/16.part',
 '/Users/home/Medium_Trans.csv.gz/17.part',
 '/Users/home/Medium_Trans.csv.gz/18.part',
 '/Users/home/Medium_Trans.csv.gz/19.part',
 '/Users/home/Medium_Trans.csv.gz/20.part',
 '/Users/home/Medium_Trans.csv.gz/21.part',
 '/Users/home/Medium_Trans.csv.g

In [24]:
file_size = os.path.getsize('Medium_Trans.csv.gz')
num_rows = len(data)
num_cols = len(list(data.columns))

# Print file summary
print("File summary:")
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")
print(f"File size: {file_size} bytes")

File summary:
Number of rows: 31251483
Number of columns: 11
File size: 1568 bytes


In [25]:
data.columns

Index(['timestamp', 'from_bank', 'account', 'to_bank', 'account_1',
       'amount_received', 'receiving_currency', 'amount_paid',
       'payment_currency', 'payment_format', 'is_laundering'],
      dtype='object')